In [1]:
%matplotlib widget

In [2]:
import numpy as np
import xarray as xr
import ipyvolume as ipv
import basic_meshes as bm
from scipy.spatial import Delaunay
from experimental_setup import get_setup
from scipy.spatial.transform import Rotation as rot

# plotting
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt

# interactive plots
import ipywidgets as widgets

from ipywidgets import VBox, HBox, IntSlider, Checkbox, interactive_output, FloatSlider
from IPython.display import display

import libo
from libo.io import tc3

import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vs

# Setup

In [3]:
setup = get_setup(5)

profile_raster_width = 4
trace_raster_width = 4

# Import and process experimental data

In [4]:
# import machine data
ds_system_data_base = tc3.to_xarray(tc3.read_db(setup.system_data_index))

# import scan data
scan_data_layer_0 = libo.io.tools.merge_scan_tcp(
    scan=setup.scan_data_layer_0_scan_idx, tcp=setup.scan_data_layer_0_tcp_idx
)
scan_data_layer_1 = libo.io.tools.merge_scan_tcp(
    scan=setup.scan_data_layer_1_scan_idx, tcp=setup.scan_data_layer_1_tcp_idx
)

# collect relevant data
ds_relevant_data = ds_system_data_base[
    ["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz", "trigSchweissen", "trigScan2_prog"]
].dropna("time")
if setup.measurement_data_id is not None:
    dsx_measurement_data = tc3.to_xarray(tc3.read_db(setup.measurement_data_id))
    dsx_temperature_data = dsx_measurement_data[["MH24_T01", "MH24_T02", "MH24_T03"]].dropna("time")
    ds_relevant_data = xr.merge([dsx_temperature_data.interp_like(ds_relevant_data), ds_relevant_data])
    scan_data_layer_0 = xr.merge([dsx_temperature_data.interp(time=scan_data_layer_0.time), scan_data_layer_0],)
    scan_data_layer_1 = xr.merge([dsx_temperature_data.interp(time=scan_data_layer_1.time), scan_data_layer_1],)


# extract system movement data
ds_movement_data = ds_relevant_data[["FB_X", "FB_Y", "FB_Z", "FB_Rx", "FB_Ry", "FB_Rz"]]
movement_coordinates_base = ds_movement_data.to_array().data[:3, :]
movement_angles_degree_base = ds_movement_data.to_array().data[3:6, :]

# extract welding movement data
ds_welding_movement_data = ds_movement_data.where(ds_system_data_base.trigSchweissen == 1)
welding_coordinates_base = ds_welding_movement_data.to_array().data[:3, :]
welding_angles_degree_base = ds_welding_movement_data.to_array().data[3:, :]

# extract scanner movement data
scanner_tcp_layer_0_coordinates_base = scan_data_layer_0.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data
scanner_tcp_layer_0_angles_degree_base = scan_data_layer_0.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data
scanner_tcp_layer_1_coordinates_base = scan_data_layer_1.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data
scanner_tcp_layer_1_angles_degree_base = scan_data_layer_1.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data

In [5]:
plt.figure()
plt.plot(scan_data_layer_1.MH24_T01.data)
scan_data_layer_0
dsx_temperature_data

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<xarray.Dataset>
Dimensions:   (time: 152051)
Coordinates:
  * time      (time) datetime64[ns] 2020-02-25T13:32:16.755000 ... 2020-02-25T13:57:37.355000
Data variables:
    MH24_T01  (time) float64 22.21 22.21 22.21 22.21 ... 28.83 28.83 28.83 28.83
    MH24_T02  (time) float64 21.97 21.97 21.97 21.97 ... 51.81 51.81 51.81 51.81
    MH24_T03  (time) float64 22.1 22.1 22.1 22.11 ... 54.48 54.48 54.48 54.48
Attributes:
    scopeName:         TC3 Export Tool
    svdFile:           D:\Messdaten\Leistungsmessung\2020_02_25_14_57_38_Leis...
    StartTimeTC3:      132271111367550000
    EndTimeTC3:        132271126573580000
    RunTime:           1520.603
    StartTime:         2020-02-25 14:32:16.755000 +0100
    EndTime:           2020-02-25 14:57:37.358000 +0100
    hostname:          WS7096
    version:           0.2.2
    TC3Import:         Python3
    filename:          
    filepath:          
    RunTime_TimeUnit:  s
    file_id:           18202

In [6]:
scan_data_layer_1

<xarray.Dataset>
Dimensions:       (n: 1280, profile: 1597, scan_coord: 3, tcp_variable: 6)
Coordinates:
    time          (profile) datetime64[ns] 2020-02-25T13:50:50.063000 ... 2020-02-25T13:51:06.053000
  * profile       (profile) uint64 1 2 3 4 5 6 ... 1595 1596 1597 1598 1599 1600
  * tcp_variable  (tcp_variable) <U2 'X' 'Y' 'Z' 'Rx' 'Ry' 'Rz'
  * scan_coord    (scan_coord) <U1 'x' 'y' 'z'
  * n             (n) uint64 0 1 2 3 4 5 6 ... 1274 1275 1276 1277 1278 1279
Data variables:
    MH24_T01      (profile) float64 352.7 352.7 352.7 ... 296.9 296.9 296.9
    MH24_T02      (profile) float64 249.1 249.1 249.1 ... 226.5 226.5 226.5
    MH24_T03      (profile) float64 410.0 410.0 410.0 ... 337.5 337.5 337.0
    scan_tcp      (profile, tcp_variable) float64 48.01 1.026e+03 ... 89.54
    scan_line     (profile, n, scan_coord) float64 0.0 54.88 ... -59.74 208.9

# Define coordinate systems

- `cs_base` is the root coordinate system
- `cs_ref` is defined in `cs_base` -> coordinate system created from 3 robot positions
- `cs_sp` is defined in `cs_ref` -> specimen coordinate system with x pointing into the direction of the trace

All data will be transformed to the specimens coordinate system. Data which is not defined in the specimen coordinate system gets the corresponing coordinate system's suffix attached to the variable name

Important note: Measured data is not perfectly orthogonal. We use the cross product to get a z axis which is perfectly orthogonal (in floating point precision) and recalculate a perfectly orthogonal y-axis. There are certainly better ways to address this problem.

In [7]:
# Calculate orthogonal z axis and construct reference coordinate system from x and z axis
vec_x = setup.offset_ox_ref - setup.origin_ref
vec_y = setup.offset_oy_ref - setup.origin_ref
vec_z = np.cross(vec_x, vec_y)
cs_ref_in_base = tf.LocalCoordinateSystem.construct_from_xz_and_orientation(
    vec_x=vec_x, vec_z=vec_z, origin=setup.origin_ref
)


orientation_scanner_tcp_in_flange = rot.from_euler(
    angles=[21.9714, -0.7410, -89.7500], seq="xyz", degrees=True
).as_dcm()
orientation_torch_tcp_in_flange = rot.from_euler(angles=[-22.0474, 0.5659, 90.7092], seq="xyz", degrees=True).as_dcm()

# create constant coordinate systems
cs_sp_in_ref = tf.LocalCoordinateSystem(setup.orientation_sp_in_ref, setup.coordinates_sp_in_ref)
cs_sp_in_base = cs_sp_in_ref + cs_ref_in_base
cs_torch_tcp_in_flange = tf.LocalCoordinateSystem(orientation_torch_tcp_in_flange, [-49.325, -0.410, 477.208])
cs_scanner_tcp_in_flange = tf.LocalCoordinateSystem(orientation_scanner_tcp_in_flange, [-47.949, -105.258, 479.690])
cs_scan_in_scanner_tcp = tf.LocalCoordinateSystem(basis=[[1, 0, 0], [0, 1, 0], [0, 0, -1]], origin=[0, 0, 260])
cs_scanner_tcp_in_torch_tcp = cs_scanner_tcp_in_flange - cs_torch_tcp_in_flange
cs_torch_tcp_in_scanner_tcp = cs_torch_tcp_in_flange - cs_scanner_tcp_in_flange
cs_temp_01_in_ref = tf.LocalCoordinateSystem(origin=setup.coordinates_temp_1_in_base)
cs_temp_02_in_ref = tf.LocalCoordinateSystem(origin=setup.coordinates_temp_2_in_base)
cs_temp_03_in_ref = tf.LocalCoordinateSystem(origin=setup.coordinates_temp_3_in_base)
cs_temp_01_in_sp = cs_temp_01_in_ref - cs_sp_in_ref
cs_temp_02_in_sp = cs_temp_02_in_ref - cs_sp_in_ref
cs_temp_03_in_sp = cs_temp_03_in_ref - cs_sp_in_ref



# create functions for variable coordinate systems
def get_cs_scanner_tcp_in_sp(scan_data, index):
    angles = scan_data.scan_tcp.sel(tcp_variable=["Rx", "Ry", "Rz"]).data[index]
    orientation = rot.from_euler(angles=angles, seq="xyz", degrees=True).as_dcm()
    coordinates = scan_data.scan_tcp.sel(tcp_variable=["X", "Y", "Z"]).data[index]
    cs_scanner_tcp_in_base = tf.LocalCoordinateSystem(basis=orientation, origin=coordinates)
    return cs_scanner_tcp_in_base - cs_ref_in_base - cs_sp_in_ref


def get_cs_scan_in_sp(scan_data, index):
    return cs_scan_in_scanner_tcp + get_cs_scanner_tcp_in_sp(scan_data, index)

# Define data transformations

In [8]:
def coordinates_to_child_from_parent(coordinates, cs_child):
    rotation = cs_child.orientation.transpose()
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates - translation)


def coordinates_to_parent_from_child(coordinates, cs_child):
    rotation = cs_child.orientation
    translation = cs_child.location[:, np.newaxis]
    return np.matmul(rotation, coordinates) + translation

# Transform data to specimen coordinate system

In [9]:
welding_coordinates_sp = coordinates_to_child_from_parent(welding_coordinates_base, cs_sp_in_base)


def transform_scan_data_to_sp(scan_data_in_scan):
    scanned_profiles_in_sp = []
    num_scans = scan_data_in_scan.profile.size
    for i in range(num_scans):
        scanned_profile_in_scan = scan_data_in_scan.scan_line.data[i].transpose()
        scanned_profiles_in_sp += [
            coordinates_to_parent_from_child(scanned_profile_in_scan, get_cs_scan_in_sp(scan_data_in_scan, i))
        ]
    return np.array(scanned_profiles_in_sp, float)


scanned_profiles_layer_0_sp = transform_scan_data_to_sp(scan_data_layer_0)
scanned_profiles_layer_1_sp = transform_scan_data_to_sp(scan_data_layer_1)

# Create theoretical geometry

In [10]:
# create points
pt_0 = [150, 8]
pt_1_1 = [np.tan(setup.groove_angle_start / 360 * np.pi) * 8, 8]
pt_1_2 = [np.tan(setup.groove_angle_end / 360 * np.pi) * 8, 8]
pt_2 = [0, 0]
pt_3 = [150, 0]

# create shapes
shape_p1_r = geo.Shape().add_line_segments([pt_0, pt_1_1, pt_2])
shape_p2_r = geo.Shape().add_line_segments([pt_0, pt_1_2, pt_2])
shape_p1_l = shape_p1_r.reflect([1, 0])
shape_p2_l = shape_p2_r.reflect([1, 0])

# create profiles
profile_1 = geo.Profile([shape_p1_l, shape_p1_r])
profile_2 = geo.Profile([shape_p2_l, shape_p2_r])

# create variable profile
variable_profile = geo.VariableProfile([profile_1, profile_2], [0, 1], [geo.linear_profile_interpolation_sbs])

# create trace
trace = geo.Trace(geo.LinearHorizontalTraceSegment(350))

# create geometry
geometry = geo.Geometry(variable_profile, trace)

# rasterize profiles
profile_1_data = profile_1.rasterize(4)
profile_2_data = profile_2.rasterize(4)

# rasterize geometry
geometry_data_sp = geometry.rasterize(profile_raster_width=profile_raster_width, trace_raster_width=trace_raster_width)

# calculate triangles
triangles = Delaunay(geometry_data_sp.transpose()[:, :2]).simplices

# Create sequence

In [11]:
# create time line

num_timesteps_scan_0 = 400
num_scans_layer_0 = scan_data_layer_0.profile.size
indices_scan_0 = np.arange(0, num_scans_layer_0 + 0.5, num_scans_layer_0 / num_timesteps_scan_0)
indices_scan_0 = np.array(np.round(indices_scan_0), int)


num_timesteps_movement = 800
idx_scan_1_start = np.argmax(ds_relevant_data.trigScan2_prog.data > 0.1)
num_movement_data = idx_scan_1_start  # movement_coordinates_base.shape[1]
indices_movement = np.arange(0, num_movement_data + 0.5, num_movement_data / num_timesteps_movement)
indices_movement = np.array(np.round(indices_movement), int)


num_timesteps_scan_1 = 400
num_scans_layer_1 = scan_data_layer_1.profile.size
indices_scan_1 = np.arange(0, num_scans_layer_1 + 0.5, num_scans_layer_1 / num_timesteps_scan_1)
indices_scan_1 = np.array(np.round(indices_scan_1), int)


num_timesteps_total = num_timesteps_scan_0 + num_timesteps_movement + num_timesteps_scan_1

# Create meshes

In [12]:
scale_scanner = 15


[vert_scanner, tri_scanner] = bm.create_cone_mesh(40)
vert_scanner = vert_scanner + np.array([0, 0, -1])[:, np.newaxis]
vert_scanner = np.matmul([[scale_scanner, 0, 0], [0, scale_scanner, 0], [0, 0, scale_scanner]], vert_scanner)


scale_torch = 15

[vert_torch, tri_torch] = bm.create_cone_mesh(40)
vert_torch = vert_torch + np.array([0, 0, -1])[:, np.newaxis]
vert_torch = np.matmul([[scale_torch, 0, 0], [0, scale_torch, 0], [0, 0, scale_torch]], vert_torch)

cylinder_radius = 5
cylinder_height = 1
[vertices_cylinder, triangle_indices_cylinder] = bm.create_unit_cylinder_mesh(40,1)
vertices_cylinder = np.matmul([[cylinder_radius, 0, 0], [0, cylinder_radius, 0], [0, 0, cylinder_height]], vertices_cylinder)

In [13]:
out2 = widgets.Output(layout={"border": "0px solid black"})
# create figure inside output widget
with out2:
    fig = plt.figure()
    fig.canvas.layout.height = "400px"
    fig.canvas.layout.width = "800px"
    gs = fig.add_gridspec(1, 1)
    ax_0 = fig.add_subplot(gs[0, 0])


ipvfig = ipv.figure()
layout = widgets.Layout(width="200px", height="40px")
style = {"description_width": "initial"}


# ipv.show()

play = widgets.Play(
    value=0, min=0, max=num_timesteps_total, step=1, interval=10, description="Press play", disabled=False
)
time_slider = IntSlider(min=0, max=num_timesteps_total, step=1, description="time", continuous_update=True)
w1 = dict(time=time_slider)

widgets.jslink((play, "value"), (time_slider, "value"))
w = {**w1}


upper_box = HBox()
lower_box = HBox()
box = VBox([upper_box, lower_box])

ipv.plot_trisurf(
    geometry_data_sp[0], geometry_data_sp[1], geometry_data_sp[2], triangles=triangles, color=[0.6, 0.6, 0.6]
)
ipv.plot_trisurf(vert_scanner[0], vert_scanner[1], vert_scanner[2], triangles=tri_scanner, color=[1, 0, 0])
ipv.plot_trisurf(vert_torch[0], vert_torch[1], vert_torch[2], triangles=tri_torch, color=[0, 1, 0])
ipv.plot_trisurf(vertices_cylinder[0], vertices_cylinder[1], vertices_cylinder[2], triangles=triangle_indices_cylinder, color=[0, 0, 1])
ipv.plot_trisurf(vertices_cylinder[0], vertices_cylinder[1], vertices_cylinder[2], triangles=triangle_indices_cylinder, color=[0, 0, 1])
ipv.plot_trisurf(vertices_cylinder[0], vertices_cylinder[1], vertices_cylinder[2], triangles=triangle_indices_cylinder, color=[0, 0, 1])
scanned_profiles = np.concatenate((scanned_profiles_layer_0_sp, scanned_profiles_layer_1_sp), axis=0)
ipv.scatter(
    scanned_profiles[:, 0, :],
    scanned_profiles[:, 1, :],
    scanned_profiles[:, 2, :],
    size=1,
    marker="sphere",
    color="red",
)

x_min = 0
x_max = 350
y_min = -175
y_max = 175
z_min = -20
z_max = 330
ipv.xlim(x_min, x_max)
ipv.ylim(y_min, y_max)
ipv.zlim(z_min, z_max)

d_x = x_max - x_min
d_y = y_max - y_min
d_z = z_max - z_min

s_x = d_x / d_y
s_y = d_y / d_y
s_z = d_z / d_y

axix_correction_scale_mat = [[s_x, 0, 0], [0, s_y, 0], [0, 0, s_z]]


cs_scanner_schematic = tf.LocalCoordinateSystem(origin=[0, 0, -260])

idx_switch_cs = np.argmax(ds_movement_data.FB_Rz.dropna("time").diff("time").data)
idx_scan_1 = np.argmax(ds_relevant_data.trigScan2_prog.data > 0.1)


def transform_and_update_mesh(vertex_data_mesh, cs_transformation, ipv_idx, scale_x=1, scale_y=1, scale_z=1):
    scale_mat = [[scale_x, 0, 0], [0, scale_y, 0], [0, 0, scale_z]]
    vertex_data_trans = np.matmul(scale_mat, vertex_data_mesh)
    vertex_data_trans = np.matmul(cs_transformation.orientation, vertex_data_trans)
    vertex_data_trans = np.matmul(axix_correction_scale_mat, vertex_data_trans)
    vertex_data_trans = vertex_data_trans + cs_transformation.location[:, np.newaxis]
    ipvfig.meshes[ipv_idx].x = vertex_data_trans[0]
    ipvfig.meshes[ipv_idx].y = vertex_data_trans[1]
    ipvfig.meshes[ipv_idx].z = vertex_data_trans[2]


def update_meshes(cs_scanner_in_sp, cs_torch_in_sp, torch_color, t_0, t_1, t_2):
    transform_and_update_mesh(vert_scanner, cs_scanner_in_sp, 1)
    transform_and_update_mesh(vert_torch, cs_torch_in_sp, 2)
    ipvfig.meshes[2].color = torch_color
    
    transform_and_update_mesh(vertices_cylinder, cs_temp_01_in_sp, 3, scale_z = t_0/2)
    transform_and_update_mesh(vertices_cylinder, cs_temp_02_in_sp, 4, scale_z = t_1/2)
    transform_and_update_mesh(vertices_cylinder, cs_temp_03_in_sp, 5, scale_z = t_2/2)
    
    color_morph_t_0 = np.clip(t_0/600, 0,1)
    color_morph_t_1 = np.clip(t_1/600, 0,1) 
    color_morph_t_2 = np.clip(t_2/600, 0,1) 
    ipvfig.meshes[3].color = [color_morph_t_0,0,1 - color_morph_t_0]
    ipvfig.meshes[4].color = [color_morph_t_1,0,1 - color_morph_t_1]
    ipvfig.meshes[5].color = [color_morph_t_2,0,1 - color_morph_t_2]


def visualize_scan_phase(scan_data, scanned_profiles_in_sp, idx, num_timesteps_scan, idx_offset=0):
    cs_scanner_tcp_in_sp = get_cs_scanner_tcp_in_sp(scan_data, idx)
    cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_sp
    cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_scanner_tcp_in_sp

    weight = cs_scanner_in_sp.location[0] / 350
    profile_local = geo.linear_profile_interpolation_sbs(profile_1, profile_2, weight)
    profile_local_data = profile_local.rasterize(20)
    num_raster_points = profile_local_data.shape[1]
    nhalf = int(num_raster_points / 2)
    ax_0.clear()
    ax_0.plot(profile_local_data[0, 1:nhalf], profile_local_data[1, 1:nhalf], "b", label="ideal profile")
    ax_0.plot(profile_local_data[0, nhalf:], profile_local_data[1, nhalf:], "b")
    ax_0.plot(scanned_profiles_in_sp[idx, 1, :], scanned_profiles_in_sp[idx, 2, :], "r", label="scan")
    ax_0.set_xlim([-20, 20])
    ax_0.set_ylim([0, 12])
    ax_0.legend()
    ipvfig.scatters[0].sequence_index = idx.item() + idx_offset
    ipvfig.scatters[0].visible = True
    t_0 = scan_data.MH24_T01.data[idx]
    t_1 = scan_data.MH24_T02.data[idx]
    t_2 = scan_data.MH24_T03.data[idx]
    update_meshes(cs_scanner_in_sp, cs_torch_in_sp, [0, 1, 0],t_0,t_1,t_2)


def update_output(time):

    if time < num_timesteps_scan_0:
        scn_idx = indices_scan_0[time]
        visualize_scan_phase(scan_data_layer_0, scanned_profiles_layer_0_sp, scn_idx, num_timesteps_scan_0)
    elif time < num_timesteps_scan_0 + num_timesteps_movement:
        ipvfig.scatters[0].visible = False
        mvm_idx = indices_movement[time - num_timesteps_scan_0]

        x_angles_base = movement_angles_degree_base[:, mvm_idx]
        x_coordinates_base = movement_coordinates_base[:, mvm_idx]
        x_orientation_base = rot.from_euler(angles=x_angles_base, seq="xyz", degrees=True).as_dcm()
        cs_x_in_base = tf.LocalCoordinateSystem(basis=x_orientation_base, origin=x_coordinates_base)
        cs_x_in_sp = cs_x_in_base - cs_ref_in_base - cs_sp_in_ref
        if mvm_idx > idx_switch_cs:
            cs_torch_in_sp = cs_torch_tcp_in_scanner_tcp + cs_x_in_sp
            cs_scanner_in_sp = cs_scanner_schematic + cs_x_in_sp
        else:
            cs_torch_in_sp = cs_x_in_sp
            cs_scanner_in_sp = cs_scanner_schematic + cs_scanner_tcp_in_torch_tcp + cs_torch_in_sp
        torch_color = [0, 1, 0]
        print("function called")
        if ds_relevant_data.trigSchweissen.data[mvm_idx] > 0:
            brightness = (time % 5) / 4
            torch_color = [brightness, brightness, 1]
            print("triggered")
        t_0 = ds_relevant_data.MH24_T01.data[mvm_idx] 
        t_1 = ds_relevant_data.MH24_T02.data[mvm_idx] 
        t_2 = ds_relevant_data.MH24_T03.data[mvm_idx] 
        update_meshes(cs_scanner_in_sp, cs_torch_in_sp, torch_color,t_0,t_1,t_2)
    else:
        scn_idx = indices_scan_1[time - num_timesteps_scan_0 - num_timesteps_movement]
        visualize_scan_phase(
            scan_data_layer_1, scanned_profiles_layer_1_sp, scn_idx, num_timesteps_scan_1, num_scans_layer_0
        )


output = interactive_output(update_output, w)
upper_box.children = [ipvfig, out2]
lower_box.children = [play, time_slider]
display(box)

In [14]:
update_output(385)
scan_data_layer_0.MH24_T01.data[200]

22.2

In [15]:
idx_switch_cs = np.argmax(ds_movement_data.FB_Rz.dropna("time").diff("time").data)
idx_switch_cs
indices_movement[293 - num_timesteps_scan_0]

8657

In [16]:
ds_relevant_data.trigSchweissen.data[2320]

1.0

In [17]:
plt.figure()
plt.plot(ds_relevant_data.trigSchweissen.data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
np.argmax(ds_relevant_data.trigScan2_prog.data > 0.1)

9979